# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

In [4]:
last_name = driver.find_element_by_xpath('//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[7]/td/p/input')
last_name.send_keys("Nguyen")

In [5]:
button = driver.find_element_by_xpath('//*[@id="dat-menu"]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()

## Scraping

Once you are on the results page, do this.

### Loop through each result and print the entire row

Okay wait, that's a heck of a lot. Use `[:10]` to only do the first ten (`listname[:10]` gives you the first ten).

In [6]:
cosmetologists = driver.find_elements_by_tag_name('tr')
for cosmetologist in cosmetologists[0:11]:
    print(cosmetologist.text)


Name and Location Order Basis for Order
NGUYEN, TOAN HUU
City: SAN ANTONIO
County: BEXAR
Zip Code: 78217


License #(s): 780948, 1706491, 1699123

Complaint # COS20180004289 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $500. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
NGUYEN, HANH CONG
City: EL PASO
County: EL PASO
Zip Code: 79934


License #: 737708

Complaint # COS20180006594 Date: 5/30/2018

Respondent is assessed an administrative penalty in the amount of $1,000. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
NGUYEN, KHIEM VAN
City: LONGVIEW
County: GREGG
Zip Code: 75604


License #: 731665

Complaint # COS20180000257 Date: 5/17/2018

Respondent is assessed an administrative penalty in the amount of $1,250. Responde

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

In [30]:
for cosmetologist in cosmetologists[1:11]:
    cos_names = cosmetologist.find_elements_by_tag_name('span')
    print(cos_names[0].text)

    

NGUYEN, TOAN HUU
NGUYEN, HANH CONG
NGUYEN, KHIEM VAN
NGUYEN, DIEP THI NGOC
NGUYEN, LAN T-THUY
NGUYEN, TUAN A
NGUYEN, THAO B
NGUYEN, BETH MARIA
NGUYEN, KENNEY TUAN
NGUYEN, TRUNG N


## Loop through each result, printing each violation description ("Basis for order")

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: You can get the HTML of something by doing `.get_attribute('innerHTML')` - it might help you diagnose your issue.*
> - *Tip: Or I guess you could just skip the one with the problem...

In [8]:
cosmetologists = driver.find_elements_by_tag_name('tr')
for cosmetologist in cosmetologists[1:11]:
    cos_names = cosmetologist.find_elements_by_tag_name('td')
    print(cos_names[2].text)

Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect all wax pots.
Respondent failed to disinfect tools, implements, and supplies with an EPA-registered disinfectant solution; Respondent failed to disinfect multi-use equipment, implements, and tools prior to use on each client.
Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use.
Respondent failed to clean and disinfect all wax pots; Respondent failed to disinfect tools, implements, an

## Loop through each result, printing the complaint number

- TIP: Think about the order of the elements

In [9]:
cosmetologists = driver.find_elements_by_tag_name('tr')
for cosmetologist in cosmetologists[1:11]:
    cos_names = cosmetologist.find_elements_by_tag_name('span')
    print(cos_names[10].text)

COS20180004289
COS20180006594
COS20180000257
COS20180004915
County:
County:
COS20180008846
COS20180000897
BAR20180001231
COS20170023893


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [13]:
cosmetologists = driver.find_elements_by_tag_name('tr')

for cosmetologist in cosmetologists[1:11]:
    cos_names = cosmetologist.find_elements_by_tag_name('span')
    cos_names[1].text

In [33]:
cosmetic_violations = []

for cosmetologist in cosmetologists[1:11]:
    cos_names = cosmetologist.find_elements_by_tag_name('span')
    
    
    cosmetic_info = {}
    cosmetic_info['Name of the person'] = cos_names[0].text
    cosmetic_info['Violation number'] = cos_names[10].text
    cosmetic_info['License Numbers'] = cos_names[8].text
    cosmetic_info['Zip Code'] = cos_names[6].text
    cosmetic_info['County'] = cos_names[4].text
    cosmetic_info['City'] = cos_names[2].text
    
    cos_names1 = cosmetologist.find_elements_by_tag_name('td')
    cosmetic_info['Violation description'] = cos_names1[2].text
    
    cosmetic_violations.append(cosmetic_info)
    
    
print(cosmetic_violations)

[{'Name of the person': 'NGUYEN, TOAN HUU', 'Violation number': 'COS20180004289', 'License Numbers': '780948, 1706491, 1699123', 'Zip Code': '78217', 'County': 'BEXAR', 'City': 'SAN ANTONIO', 'Violation description': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day.'}, {'Name of the person': 'NGUYEN, HANH CONG', 'Violation number': 'COS20180006594', 'License Numbers': '737708', 'Zip Code': '79934', 'County': 'EL PASO', 'City': 'EL PASO', 'Violation description': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day; Respondent failed to use items subject to possible cross contamination in a manner that does not contaminate the remaining product.'}, {'Name of the person': 'NGUYEN, KHIEM VAN', 'Violation number': 'COS20180000257', 'License Numbers': '731665', 'Zip Code': '75604', 'County': 'GREGG', 'City': 'LONGVIEW', 'Violation description': 'Respondent failed to follow whirlpool foot spas cleaning 

### Save that to a CSV

- Tip: You'll want to use pandas here

In [34]:
import pandas as pd

In [35]:
df = pd.DataFrame(cosmetic_violations)
df.head()

,City,County,License Numbers,Name of the person,Violation description,Violation number,Zip Code
0,SAN ANTONIO,BEXAR,"780948, 1706491, 1699123","NGUYEN, TOAN HUU",Respondent failed to clean and sanitize whirlp...,COS20180004289,78217
1,EL PASO,EL PASO,737708,"NGUYEN, HANH CONG",Respondent failed to clean and sanitize whirlp...,COS20180006594,79934
2,LONGVIEW,GREGG,731665,"NGUYEN, KHIEM VAN",Respondent failed to follow whirlpool foot spa...,COS20180000257,75604
3,HOUSTON,HARRIS,"1347649, 760528","NGUYEN, DIEP THI NGOC","Respondent failed to disinfect tools, implemen...",COS20180004915,77014
4,SAN ANTONIO,BEXAR,City:,"NGUYEN, LAN T-THUY","Respondent failed to clean, disinfect, and ste...",County:,78255


In [36]:
df = df[['Name of the person', 'Violation description', 'Violation number', 'License Numbers', 'Zip Code', 'County', 'City']]
df.head()

,Name of the person,Violation description,Violation number,License Numbers,Zip Code,County,City
0,"NGUYEN, TOAN HUU",Respondent failed to clean and sanitize whirlp...,COS20180004289,"780948, 1706491, 1699123",78217,BEXAR,SAN ANTONIO
1,"NGUYEN, HANH CONG",Respondent failed to clean and sanitize whirlp...,COS20180006594,737708,79934,EL PASO,EL PASO
2,"NGUYEN, KHIEM VAN",Respondent failed to follow whirlpool foot spa...,COS20180000257,731665,75604,GREGG,LONGVIEW
3,"NGUYEN, DIEP THI NGOC","Respondent failed to disinfect tools, implemen...",COS20180004915,"1347649, 760528",77014,HARRIS,HOUSTON
4,"NGUYEN, LAN T-THUY","Respondent failed to clean, disinfect, and ste...",County:,City:,78255,BEXAR,SAN ANTONIO


### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [37]:
df.to_csv("texas-cosmetology.csv", index=False)

In [38]:
df = pd.read_csv("texas-cosmetology.csv")

In [39]:
df

,Name of the person,Violation description,Violation number,License Numbers,Zip Code,County,City
0,"NGUYEN, TOAN HUU",Respondent failed to clean and sanitize whirlp...,COS20180004289,"780948, 1706491, 1699123",78217,BEXAR,SAN ANTONIO
1,"NGUYEN, HANH CONG",Respondent failed to clean and sanitize whirlp...,COS20180006594,737708,79934,EL PASO,EL PASO
2,"NGUYEN, KHIEM VAN",Respondent failed to follow whirlpool foot spa...,COS20180000257,731665,75604,GREGG,LONGVIEW
3,"NGUYEN, DIEP THI NGOC","Respondent failed to disinfect tools, implemen...",COS20180004915,"1347649, 760528",77014,HARRIS,HOUSTON
4,"NGUYEN, LAN T-THUY","Respondent failed to clean, disinfect, and ste...",County:,City:,78255,BEXAR,SAN ANTONIO
5,"NGUYEN, TUAN A",Respondent failed to clean and disinfect all w...,County:,City:,78723,TRAVIS,AUSTIN
6,"NGUYEN, THAO B",Respondent failed to clean and sanitize whirlp...,COS20180008846,"721373, 1142884",76039,TARRANT,EULESS
7,"NGUYEN, BETH MARIA",The Respondent's license was revoked upon Resp...,COS20180000897,1470271,77083,HARRIS,HOUSTON
8,"NGUYEN, KENNEY TUAN",Respondent leased space in a barber shop to an...,BAR20180001231,692892,78613,WILLIAMSON,CEDAR PARK
9,"NGUYEN, TRUNG N","Respondent failed to clean, disinfect, and ste...",COS20170023893,"1196244, 767015, 767014",79106,POTTER,AMARILLO
